In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -U scikit-learn -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, mean_squared_error, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.base import copy
import pandas as pd
import numpy as np
import random
import optuna

In [4]:
RANDOM_SEED = 42
FIND_BEST_PARAMS = False


np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [5]:
train   = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
orginal = pd.read_csv('/kaggle/input/abalone-dataset/abalone.csv')
test    = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')

In [6]:
train = train.drop(['id'], axis = 1)
train.columns = orginal.columns
train = pd.concat([train, orginal], axis = 0, ignore_index=True)

y = train['Rings']
y_log = np.log(1+y)

train = train.drop(['Rings'], axis = 1)
train.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975


In [7]:
test_id = test['id']
test = test.drop('id', axis = 1)
test.columns = train.columns
test.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050


In [8]:
encoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

train = pd.concat([
                    train.iloc[:,1:], 
                    pd.DataFrame(encoder.fit_transform(train[['Sex']]).astype('int'), 
                                 columns = encoder.categories_[0])
                    ], 
                    axis = 1
                )

test  = pd.concat([
                    test.iloc[:,1:], 
                    pd.DataFrame(encoder.transform(test[['Sex']]).astype('int'), 
                                 columns = encoder.categories_[0])
                    ], 
                    axis = 1
                )

In [9]:
%%time


def objective(trial):

    params = {
        "verbose": False,
        "iterations": 1000,
        "loss_function":'RMSE',
        "random_state": RANDOM_SEED,
        "depth": trial.suggest_int("depth", 3, 15),
        "subsample": trial.suggest_float("subsample", 0.01, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    scores = []
    for _, (train_index, valid_index) in enumerate(cv.split(train, y)):
        X_train, y_train = train.iloc[train_index], y_log.iloc[train_index]
        X_valid, y_valid = train.iloc[valid_index], y_log.iloc[valid_index]
        model = CatBoostRegressor(**params)

        model.fit(X_train, y_train, 
                  eval_set=(X_valid, y_valid),
                  early_stopping_rounds=100)
        
        y_pred = model.predict(X_valid)
        scores.append(root_mean_squared_error(y_valid, y_pred))
    return np.mean(scores)


study = optuna.create_study(direction='minimize', study_name="optuna_catboost")
if FIND_BEST_PARAMS:
    study.optimize(objective, n_trials=50)
    print(f"Best trial average RMSE: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

[I 2024-04-07 13:39:19,731] A new study created in memory with name: optuna_catboost


CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.11 ms


In [10]:
%%time


def objective(trial):

    params = {
        'n_jobs':-1,
        "metric":'rmse',  
        "verbosity": -1,
        "bagging_freq": 1,
        "boosting_type": "gbdt",    
        "objective":'regression', 
        'random_state':RANDOM_SEED,
        'max_depth': trial.suggest_int('max_depth', 3, 15),                        
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "n_estimators": trial.suggest_int('n_estimators', 400, 1000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),               
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 60),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    scores = []
    for _, (train_index, valid_index) in enumerate(cv.split(train, y)):
        X_train, y_train = train.iloc[train_index], y_log.iloc[train_index]
        X_valid, y_valid = train.iloc[valid_index], y_log.iloc[valid_index]
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)     
        scores.append(root_mean_squared_error(y_valid, y_pred))
    return np.mean(scores)


study = optuna.create_study(direction='minimize', study_name="optuna_lgbm")
if FIND_BEST_PARAMS:
    study.optimize(objective, n_trials=50)
    print(f"Best trial average RMSE: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

[I 2024-04-07 13:39:19,756] A new study created in memory with name: optuna_lgbm


CPU times: user 1.07 ms, sys: 14 µs, total: 1.08 ms
Wall time: 965 µs


In [11]:


def objective(trial):

    params = {
        'eval_metric': 'rmse',
        'random_state': RANDOM_SEED,
        'objective': 'reg:squarederror',
        'gamma': trial.suggest_float("gamma", 1e-2, 1.0),
        'max_depth': trial.suggest_int('max_depth',2, 20),
        'subsample': trial.suggest_float("subsample", 0.05, 1.0),
        'n_estimators': trial.suggest_int('n_estimators',100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight',2, 20),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.05, 1.0),
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    scores = []
    for _, (train_index, valid_index) in enumerate(cv.split(train, y)):
        X_train, y_train = train.iloc[train_index], y_log.iloc[train_index]
        X_valid, y_valid = train.iloc[valid_index], y_log.iloc[valid_index]
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)     
        scores.append(root_mean_squared_error(y_valid, y_pred))
    return np.mean(scores)


study = optuna.create_study(direction='minimize', study_name="optuna_xgboost")
if FIND_BEST_PARAMS:
    study.optimize(objective, n_trials=50)
    print(f"Best trial average RMSE: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")

[I 2024-04-07 13:39:19,782] A new study created in memory with name: optuna_xgboost


CPU times: user 1.21 ms, sys: 0 ns, total: 1.21 ms
Wall time: 1.08 ms


In [12]:
xgboost_params = {
    'verbosity': 0,
    'max_depth': 10, 
    'device': 'cuda', 
    'booster': 'dart', 
    'eval_metric': 'rmsle', 
    'random_state':RANDOM_SEED,
    'lambda': 0.456836886068415, 
    'alpha': 0.6422509164613671, 
    'subsample': 0.8365423486036913, 
    'objective': 'reg:squaredlogerror', 
    'learning_rate': 0.09884907639400813, 
    'colsample_bytree': 0.8111849113860014, 
}

catboost_params = {
    'depth': 15, 
    'max_bin': 464, 
    'verbose': False,
    'task_type': 'GPU', 
    'eval_metric': 'RMSE', 
    'min_data_in_leaf': 78, 
    'loss_function': 'RMSE', 
    'grow_policy': 'Lossguide', 
    'random_state':RANDOM_SEED,
    'bootstrap_type': 'Bernoulli', 
    'subsample': 0.83862137638162, 
    'l2_leaf_reg': 8.365422739510098, 
    'random_strength': 3.296124856352495, 
    'learning_rate': 0.09992185242598203, 
}

lgbm_params = {
    'verbosity': -1,
    'device': 'gpu', 
    'metric': 'rmse', 
    'num_leaves': 176, 
    'bagging_freq': 7, 
    'boosting_type': 'gbdt', 
    'min_child_samples': 91, 
    'objective': 'regression', 
    'random_state':RANDOM_SEED,
    'learning_rate': 0.07351805347801958, 
    'bagging_fraction': 0.6502062728410578, 
    'feature_fraction': 0.7058843944694884, 
}

In [13]:
cv_estimators = [
    ('lgbm', LGBMRegressor(**lgbm_params)),
    ('xgboost', XGBRegressor(**xgboost_params)),
    ('catboost', CatBoostRegressor(**catboost_params))
]

In [14]:
%%time


def objective(trial):
    
    params = {
        'lgbm_weight': trial.suggest_float('lgbm_weight', 0.0, 5.0),
        'xgboost_weight': trial.suggest_float('xgboost_weight', 0.0, 5.0),
        'catboost_weight': trial.suggest_float('catboost_weight', 0.0, 5.0),
    }


    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    scores = []
    for _, (train_index, valid_index) in enumerate(cv.split(train, y)):
        X_train, y_train = train.iloc[train_index], y_log.iloc[train_index]
        X_valid, y_valid = train.iloc[valid_index], y_log.iloc[valid_index]
        voting_regressor = VotingRegressor(
            estimators=cv_estimators,
            weights=[params['lgbm_weight'], params['xgboost_weight'], params['catboost_weight']]
        )
        voting_regressor.fit(X_train, y_train)
        y_pred = voting_regressor.predict(X_valid)  
        scores.append(root_mean_squared_error(y_valid, y_pred))
    return np.mean(scores)


study = optuna.create_study(direction='minimize', study_name="voting_regressor_optuna")
if FIND_BEST_PARAMS:
    study.optimize(objective, n_trials=100)
    print(f"Best trial average RMSE: {study.best_value:.4f}")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")


[I 2024-04-07 13:39:19,850] A new study created in memory with name: voting_regressor_optuna


CPU times: user 943 µs, sys: 0 ns, total: 943 µs
Wall time: 813 µs


In [15]:
%%time

weight_best_params = {
    'lgbm_weight': 2.704098705074571, 
    'xgboost_weight': 0.7975293054707728, 
    'catboost_weight': 4.383257916855633,
}


voting_regressor = VotingRegressor(
    estimators=cv_estimators,
    weights=[ weight_best_params['lgbm_weight'], 
              weight_best_params['xgboost_weight'], 
              weight_best_params['catboost_weight']
    ]
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
scores = []
y_pred_test = []
for fold_i, (train_index, valid_index) in enumerate(cv.split(train, y)):
    X_train, y_train = train.iloc[train_index], y_log.iloc[train_index]
    X_valid, y_valid = train.iloc[valid_index], y_log.iloc[valid_index]
    voting_regressor.fit(X_train, y_train)
    y_pred = voting_regressor.predict(X_valid)  
    score = root_mean_squared_error(y_valid, y_pred)
    scores.append(score)
    y_pred_test.append(voting_regressor.predict(test))
    print(f"FOLD {fold_i} Done. RMSE : {score}")
print(f"All FOLD. Mean RMSE : {np.mean(scores)}")

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


FOLD 0 Done. RMSE : 0.14865781035564574
FOLD 1 Done. RMSE : 0.14724672256202875
FOLD 2 Done. RMSE : 0.14838241995366852
FOLD 3 Done. RMSE : 0.14947514500541953
FOLD 4 Done. RMSE : 0.14826518169395217
All FOLD. Mean RMSE : 0.14840545591414295
CPU times: user 2min 19s, sys: 16.3 s, total: 2min 35s
Wall time: 1min 32s


In [16]:
predictions = np.mean(y_pred_test, axis=0)
sub  = pd.DataFrame(columns = ['id', 'Rings'])
sub['id'] = test_id
sub['Rings'] = np.exp(predictions)-1
sub.to_csv('submission.csv', index = False)